### Imports

In [1]:
import torch
import torch.nn as nn
import numpy as np

import scripts.data_loader as data_loader
import scripts.model.training as training
import scripts.model.evaluation as evaluation

from models.model1 import OB_05Model

# Training

In [ ]:
training_dataset, validation_dataset, testing_dataset = data_loader.split_images_dataset()
torch.save(training_dataset, "models/training_dataset.pth")
torch.save(validation_dataset, "models/validation_dataset.pth")
torch.save(testing_dataset, "models/testing_dataset.pth")

training_set_loader = data_loader.create_data_loader(training_dataset)
validation_set_loader = data_loader.create_data_loader(validation_dataset)
testing_set_loader = data_loader.create_data_loader(testing_dataset)

In [ ]:
learning_rate = 0.001
model = OB_05Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_config = training.TrainingConfig(
    training_set_loader=training_set_loader,
    validation_set_loader=validation_set_loader,
    testing_set_loader=testing_set_loader,

    epochs=10,
    learning_rate=learning_rate,

    classes=data_loader.get_trainset().classes,
    model=model,
    criterion=criterion,
    optimizer=optimizer
)

In [ ]:
training.train_model(training_config)

torch.save(model.state_dict(), 'models/model.pth')

# Evaluation/Testing

In [2]:
loaded_testing_dataset = torch.load('models/testing_dataset.pth')
testing_set_loader = data_loader.create_data_loader(loaded_testing_dataset)

print(next(iter(loaded_testing_dataset)))

(tensor([[[ 1.7396,  1.9942,  1.9687,  1.9432,  1.4468,  0.8613,  0.6195,
           0.5049,  0.3649,  0.3013,  0.2758,  0.0849,  0.0467,  0.2504,
           0.2886,  0.3013,  0.2631,  0.1995,  0.1613,  0.3268,  0.6704,
           1.2814,  1.8032,  2.1596,  2.0960,  2.2105,  2.2233,  2.1978],
         [ 2.3251,  2.4524,  2.4015,  2.1087,  1.5105,  0.8104,  0.4286,
           0.2504,  0.1740,  0.2758,  0.3140,  0.1995,  0.1104,  0.2377,
           0.2249,  0.1486,  0.0467,  0.0213,  0.0340,  0.1486,  0.2886,
           0.7468,  1.1795,  1.8796,  2.1851,  2.3505,  2.3251,  2.2996],
         [ 2.6560,  2.6306,  2.4269,  1.7269,  1.2050,  0.6322,  0.2249,
           0.2249,  0.3013,  0.5177,  0.4668,  0.4540,  0.2504,  0.1995,
           0.0595, -0.0806, -0.0806, -0.1060, -0.0806,  0.0849,  0.0722,
           0.4031,  0.7977,  1.4214,  2.4778,  2.6433,  2.5542,  2.3251],
         [ 2.7451,  2.6687,  2.2996,  1.6632,  0.9250,  0.4922,  0.3140,
           0.3904,  0.5686,  0.7722,  0.7595,  

In [3]:
model = OB_05Model()
model.load_state_dict(torch.load('models/model.pth'))
evaluation_result = evaluation.evaluate_model(model, testing_set_loader)

In [4]:
true_positives, false_positives, true_negatives, false_negatives = evaluation_result.calculate_metrics()

print(f'Total:\t\t\t\t{evaluation_result.total_tests()} tests\n'
      f'True positives:\t\t{np.sum(true_positives)}\n'
      f'False positives:\t{np.sum(false_positives)}\n'
      f'True negatives:\t\t{np.sum(true_negatives)}\n'
      f'False negatives:\t{np.sum(false_negatives)}\n'
      f'Precision:\t\t\t{evaluation_result.calculate_precision():.4f}\n'
      f'Recall:\t\t\t\t{evaluation_result.calculate_recall():.4f}\n'
      f'Accuracy:\t\t\t{evaluation_result.calculate_accuracy():.4f}\n')


Total:				348 tests
True positives:		275
False positives:	73
True negatives:		971
False negatives:	73
Precision:			0.7902
Recall:				0.7902
Accuracy:			0.8951

